In [893]:
import lightgbm as lgb, catboost as cb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

import os
import networkx as nx
from collections import Counter

from tqdm import tqdm
import pickle

pd.set_option('display.max_columns', None)



In [894]:
# from google.colab import drive
# drive.mount('/content/drive')
# root_dir = "/kaggle/input/nsu-jobs/"

In [895]:
!pip install pymorphy2
!pip install cytoolz

In [896]:
X_train = pd.read_csv("/kaggle/input/nsu-jobs/nsu-bda-2023-jobs/X_train.csv")
y_train = pd.read_csv("/kaggle/input/nsu-jobs/nsu-bda-2023-jobs/y_train.csv")
X_train=X_train.merge(y_train,how='left', on='id')

In [897]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        27934 non-null  int64  
 1   name                      27934 non-null  object 
 2   has_test                  27934 non-null  bool   
 3   response_letter_required  27934 non-null  bool   
 4   salary_from               23902 non-null  float64
 5   salary_currency           27934 non-null  object 
 6   salary_gross              27786 non-null  object 
 7   published_at              27934 non-null  object 
 8   created_at                27934 non-null  object 
 9   employer_name             27934 non-null  object 
 10  description               27933 non-null  object 
 11  area_id                   27934 non-null  int64  
 12  area_name                 27934 non-null  object 
 13  salary_to                 27934 non-null  float64
dtypes: boo

In [898]:
X_train.description = X_train.description.fillna('')
X_train.published_at = pd.to_datetime(X_train.published_at, errors='raise')
X_train.created_at = pd.to_datetime(X_train.created_at, errors='raise')
X_train['timestamp'] = (X_train.published_at.astype('int64') / 10**9)
X_train.salary_gross=X_train.salary_gross.astype('bool')

# Features from title

In [899]:
import nltk
nltk.download('stopwords')

def int_to_roman(x):
    """
    Normalizing titles like software engineer 3

    Also filters out numbers that are not likely part of a seniority description, i.e. 2000
    """

    if not x.isnumeric():
        return x
    x = int(x)
    ints = (1000, 900,  500, 400, 100,  90, 50,  40, 10,  9,   5,  4,   1)
    nums = ('M',  'CM', 'D', 'CD','C', 'XC','L','XL','X','IX','V','IV','I')
    result = []
    for i in range(len(ints)):
        count = int(x / ints[i])
        result.append(nums[i] * count)
        x -= ints[i] * count
    result = ''.join(result).lower()
    if any([n in result for n in ['M', 'C', 'D', 'X', 'L']]):
        return ""
    return ''.join(result).lower()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [900]:
from pymorphy2 import MorphAnalyzer
from nltk import wordpunct_tokenize
from cytoolz import memoize
import re
from nltk.corpus import stopwords
from string import punctuation

morph = MorphAnalyzer()

stops = set(stopwords.words('russian'))
stops.update(set(punctuation))
stopwords = ['ул', 'пр', 'кт']
stops.update(set(stopwords))


functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words

def preprocess_title(title: str):
    if pd.isna(title) or title == "":
        return []
    tokens = wordpunct_tokenize(title)
    tokens = [token.lower() for token in tokens]
    tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]
    # tokens = [re.sub(r"\d+", "", token) for token in tokens]
  ## lemmatization
    tokens = [morph.normal_forms(token)[0] for token in tokens]

  # удалить дубликаты
    tokens = [k for k, v in Counter(tokens).items() if v == 1]

  # удалить междометия, частицы, союзы и предлоги
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
    tokens = [token for token in tokens if morph.parse(token)[0].tag.POS not in functors_pos]

  # удалить слова из 1 символа кроме
    good_words = {'c'}
    tokens = [token for token in tokens if len(token) >= 2 or token in good_words]

  #remove stop words
    tokens = [token for token in tokens if token not in stops]

    tokens = [int_to_roman(token) for token in tokens]
    tokens = list(filter(bool, tokens))
  # print(f"{title} -> {tokens}")
    return tokens

In [901]:
titles = ['директора 2', 'дата-аналитик', 'Менеджер по продажам (брокер) элитной городской недвижимости', 'Продавец-кассир (Москва, Вернадского, 93)']
for t in titles:
    print(preprocess_title(t))

['директор']
['дата', 'аналитик']
['менеджер', 'продажа', 'брокер', 'элитный', 'городской', 'недвижимость']
['продавец', 'кассир', 'москва', 'вернадский', 'xciii']


In [902]:
from tqdm import tqdm
names1 = []
names2 = []
names3 = []
for t in tqdm(X_train.name):
    title = preprocess_title(t)
    if(len(title)>=3):
        names1.append(title[0])
        names2.append(title[1])
        names3.append(title[2])
    elif(len(title)>=2):
        names1.append(title[0])
        names2.append(title[1])
        names3.append('')
    elif(len(title)>=1):
        names1.append(title[0])
        names2.append('')
        names3.append('')
    else:
        names1.append('')
        names2.append('')
        names3.append('')

100%|██████████| 27934/27934 [01:07<00:00, 413.65it/s]


In [903]:
X_train['start_name1'] = names1
X_train['start_name2'] = names2
X_train['start_name3'] = names3

In [904]:
title_counts = Counter(X_train['name'].apply(lambda x: " ".join(preprocess_title(x))).values)
del title_counts['']
title_counts.most_common(10)

[('менеджер продажа', 344),
 ('менеджер работа клиент', 203),
 ('уборщица уборщик', 197),
 ('юрист', 163),
 ('помощник юрист', 149),
 ('аналитик', 146),
 ('оператор call центр', 136),
 ('frontend разработчик', 124),
 ('копирайтер', 115),
 ('офис менеджер', 108)]

In [905]:
len(title_counts)

14968

In [906]:
from cytoolz import isdistinct, topk
from itertools import product
from operator import itemgetter

def get_gram_counts(tokens, best_grams):
    results = []
    seen = set()
    if not best_grams:
        gram_product = product(tokens, repeat=2)
    else:
        gram_product = product(tokens, best_grams)
    for title_grams in gram_product:
        if not isdistinct(title_grams):
            continue
        title = " ".join(title_grams)
        if title in seen:
            continue
        else:
            seen.add(title)
        count = title_counts.get(title, 0)
        results.append((title, count))
    return sorted(results, key=itemgetter(1), reverse=True)


def optimize_title(x:str, topn=3, title_counts=title_counts):
    tokens = preprocess_title(x)
    if not tokens:
        return x
    if len(tokens)==1:
        return tokens[0]

    starting_score = title_counts.get(x, 1)
    best_ngrams = [(token, title_counts.get(token, 0)) for token in tokens]
    gram_counter = 2
    while gram_counter <= len(tokens): # Continue chaining tokens to get the highest score
        gram_counts = get_gram_counts(tokens, [token for token, score in best_ngrams])
        best_ngrams.extend(gram_counts)
        best_ngrams = list(topk(topn, best_ngrams, key=itemgetter(1)))
        if not any([g in best_ngrams for g in gram_counts]):  # The most recent get_gram_counts did not 'make the cut'
            best_ngram_found = topk(1, best_ngrams, key=itemgetter(1))[0]
            best_ngram, best_ngram_score = best_ngram_found
            if best_ngram_score > starting_score:
                return best_ngram
            else:
                return x
        gram_counter += 1
    return best_ngrams[0][0]

In [907]:
optimum_titless = []
optimum_countss = []

for title in tqdm(X_train.name):
    optimum_title = optimize_title(title, title_counts=title_counts)
    optimum_count = title_counts[optimum_title]
    optimum_titless.append(optimum_title)
    optimum_countss.append(optimum_count)

100%|██████████| 27934/27934 [01:14<00:00, 372.80it/s]


In [908]:
X_train['optimum_title'] = optimum_titless
X_train[['name','start_name1','start_name2','start_name3','optimum_title']].sample(10)

,name,start_name1,start_name2,start_name3,optimum_title
26145,Менеджер по продаже мебели,менеджер,продажа,мебель,менеджер продажа
6878,Менеджер по продажам программного обеспечения,менеджер,продажа,программный,менеджер продажа
3533,Помощник секретаря,помощник,секретарь,,помощник
15211,Преподаватель Английского языка в Онлайн-Школе...,преподаватель,онлайн,школа,преподаватель
8068,Менеджер по работе с поставщиками,менеджер,работа,поставщик,менеджер
1746,Работа в Польше для граждан России.,работа,польша,гражданин,Работа в Польше для граждан России.
17178,HTML-Верстальщик / Frontend-разработчик,html,верстальщик,frontend,frontend разработчик
21968,"Продавец-кассир (Москва, Палехская улица, 131)",продавец,кассир,москва,продавец кассир
22128,Специалист по обеспечению качества,специалист,обеспечение,качество,специалист
23943,"Старший продавец-кассир (Москва, Щёлковское, 96)",старший,продавец,кассир,продавец кассир


# Features description

In [910]:
from pymorphy2 import MorphAnalyzer
from nltk import wordpunct_tokenize, word_tokenize
from cytoolz import memoize
import re
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter

morph = MorphAnalyzer()

stops = set(stopwords.words('russian'))
stops.update(set(punctuation))
stopwords = ['ул', 'пр', 'кт']
stops.update(set(stopwords))

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words

def is_rus(s: str):
    rus = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя1234567890')
    return all(map(lambda symbol: symbol in rus, s))

def tokenize(desc:str,is_lemmatization=False, mod=1):
    if pd.isna(desc) or desc == "":
        return []
    
    if mod == 1:
        tokens = wordpunct_tokenize(desc)
    else:
        tokens = word_tokenize(desc)
    
    tokens = [token.lower() for token in tokens]
    
    if is_lemmatization:
        tokens = [morph.normal_forms(token)[0] for token in tokens]
    
    return tokens
        
    
def desc_to_tokens(tokens:list, delete_non_unction_words=False, only_words=True):

  if only_words:
      tokens = [re.sub(r"[^\w\s]", "", token) for token in tokens]
  # tokens = [re.sub(r"\d+", "", token) for token in tokens]
  # lemmatization не делаю, т.к. высшее становится высокий
  #   if is_lemmatization:
  #       tokens = [morph.normal_forms(token)[0] for token in tokens]

  # удалить дубликаты
  # tokens = [k for k, v in Counter(tokens).items() if v == 1]

  #   удалить междометия, частицы, союзы и предлоги
  if delete_non_unction_words:
      functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}  # function words
      tokens = [token for token in tokens if morph.parse(token)[0].tag.POS not in functors_pos]

  # удалить слова из 1 символа кроме
  # good_words = {'c', '.'}
  # tokens = [token for token in tokens if len(token) >= 2 or token in good_words]

  # remove stop words
  # tokens = [token for token in tokens if token not in stops]

  tokens = list(filter(bool, tokens))
  # print(f"{desc} -> {tokens}")
  return tokens

In [911]:
"""
Проверяет, есть ли токен X в delta токенов окресности списка токенов 
m - объект match
"""
def is_word_near_eng(x: str, spisok: list, position: int, delta=6):
    
    if position >= len(spisok):
        return False
    
    okresnost = (spisok[:position])[-delta:] + (spisok[position+1:])[:delta]
    
    return x in okresnost

"""
Возвращает строку из eng_levels = ['A', 'B', 'C'] или None
"""
def get_english(desc_tokens: list):
    
    eng_levels = ['A', 'B', 'C']
    
    eng_levels_matching_strong = {
        'с1': eng_levels[2],
        'с2': eng_levels[2],
        'c1': eng_levels[2],
        'c2': eng_levels[2],
        'b1': eng_levels[1],
        'b2': eng_levels[1],
        'a1': eng_levels[0],
        'a2': eng_levels[0],
        'intermediate': eng_levels[1],
        'advanced': eng_levels[2],
        'proficient': eng_levels[2],
        'elementary': eng_levels[0],
    }
    
    eng_levels_matching = {
        'разговорный': eng_levels[2],
        'свободный': eng_levels[2],
        'свободно': eng_levels[2],
        'письменный': eng_levels[1],
        'уверенный':eng_levels[1],
        'чтение': eng_levels[1], 
        'технический': eng_levels[1],
        'литература': eng_levels[1],
        'средний': eng_levels[1],
        'хороший': eng_levels[1],
        'написание': eng_levels[1],
        'знать': eng_levels[1],
        'уровень': eng_levels[1],
        'общение': eng_levels[2],
    }
    
    angliiskii = 'английский'
    
    for token in desc_tokens:
        for eng_level in eng_levels_matching_strong.keys():
            if eng_level in token:
                return eng_levels_matching_strong[eng_level]
    
    for i in range(len(desc_tokens)):
        if angliiskii in desc_tokens[i]:
            for eng_level in eng_levels_matching.keys():
                if is_word_near_eng(position=i, spisok=desc_tokens,x=eng_level):
                    return eng_levels_matching[eng_level]
     
    return None

In [912]:
eng_test = ['Английский или немецкий язык на разговорном уровне ',
'Английский за счет компании;',
'Английский язык (письменный профессиональный must have),',
'если знаете английский язык и управляли небольшой командой.',
'Уверенный английский для изучения научной литературы.',
'английский язык - чтение технической литературы',
'Желательно английский язык для чтения тех. Документации',
'Английский не ниже Intermediate.',
'английский (средний, профессиональная терминология);',
'Команда русскоговорящая, но хороший английский может Вам пригодиться',
'Английский язык (свободно).',
'Технический английский язык (чтение документации',
'Английский язык - на уровне, достаточном для',
'английский будет плюсом',
'ной язык общения — английский',
'Английский язык не ниже уровня Upper Intermediate.',
'Английский язык Intermediate и выше',
'Корпоративный английский язык',
'Английский от upper-intermediate',
'нужно знать английский язык, а также',
'Английский язык от <strong>Intermediate<',
'Английский разговорный',
'Английский язык технический',
'Английский язык (на уровне чтения технической документации)',
'Свободный английский язык',
'Английский язык - intermediate',
'Английский не -ниже уровня B2 (Upper-Intermediate);',
'Английский, письменный и устный, свободное общение<',
'Хороший технический и разговорный английский (Intermediate +)<',
'Английский язык на уровне чтения профессиональной литературы',
'Английский язык на уровне написания текстов.',
'Разговорный английский язык.',
'Английский язык - не ниже С1;']

# h = [desc_to_tokens(a, True, True, mod = 0) for a in eng_test]

tokens_eng=[tokenize(a,is_lemmatization=True,mod = 0) for a in eng_test]
h = [desc_to_tokens(a, True) for a in tokens_eng]
h1 = [get_english(t) for t in h]

for lvl, desc in zip(h1, eng_test):
    print (f'{desc}\n -> {lvl}')

Английский или немецкий язык на разговорном уровне 
 -> C
Английский за счет компании;
 -> None
Английский язык (письменный профессиональный must have),
 -> B
если знаете английский язык и управляли небольшой командой.
 -> B
Уверенный английский для изучения научной литературы.
 -> B
английский язык - чтение технической литературы
 -> B
Желательно английский язык для чтения тех. Документации
 -> B
Английский не ниже Intermediate.
 -> B
английский (средний, профессиональная терминология);
 -> B
Команда русскоговорящая, но хороший английский может Вам пригодиться
 -> B
Английский язык (свободно).
 -> C
Технический английский язык (чтение документации
 -> B
Английский язык - на уровне, достаточном для
 -> B
английский будет плюсом
 -> None
ной язык общения — английский
 -> C
Английский язык не ниже уровня Upper Intermediate.
 -> B
Английский язык Intermediate и выше
 -> B
Корпоративный английский язык
 -> None
Английский от upper-intermediate
 -> B
нужно знать английский язык, а также
 ->

In [913]:
"""
Проверяет, есть ли токен X в delta токенов окресности списка токенов 
m - объект match
"""
def is_word_near_exp(x: str, spisok: list, position: int, delta=6):
    
    if position >= len(spisok):
        return False
    
    okresnost = (spisok[:position])[-delta:] + (spisok[position+1:])[:delta]
    
    return x in okresnost

def get_number(t: str):
    regular_digit = r'\b\d-й\b'
    if t.replace(',','',1).isdigit() or t.replace('.','',1).isdigit():
        return float(t.replace(',','.',1))           
    if re.match(regular_digit, t):  # 1-й 3-й 2-й
        return int(''.join(filter(str.isdigit, t)))
    return None
    

def get_experience(desc_tokens: list, debug=False):
#     опыт + число + год/месяц (рядом)
#     опыт + год
#     опыт работа

    year_time = ['год', 'лет']
    mounth_time = ['месяц']
    
    need_exp = ['только', 'обязательный']
    no_exp = 'без'
    
    delta = 15
    
    default_exp = 1
    
    is_need_exp = 0
    
    for i in range(len(desc_tokens)):
        if 'опыт' in desc_tokens[i]: # нашел опыт
            if debug:
                print(f'check: {desc_tokens[i+1:i+1+delta]}')
        
            next_tokens = desc_tokens[i+1:i+1+delta]
            for j in range(len(next_tokens)):
                if debug:
                    print(f'check: {next_tokens[j]}')
                num = get_number(next_tokens[j]) # в следующих токенах есть число
                if debug:
                    print(f'num: {num}')
                if num:
                    if num >= 15: # не то
                        continue
                    try:
                        if debug:
                            print(f'desc_tokens[j+1]: {next_tokens[j+1]}')
                        # после числа должен быть год или месяц

                        if next_tokens[j+1] in year_time: 
                            # опыт + число + год/месяц (рядом)
                            return num
                        if next_tokens[j+1] in mounth_time:
                            return num/12
                    except Exception:
                        continue
                else:
                    if debug:
                        print(f'{next_tokens[j]} - не число')
                    
            # в следующих токенах нет числа, поищем слово год
            for value in desc_tokens[i+1:i+1+delta]:
                if 'год' in value:
                    return 1
        
            # вероятно, это не тот опыт.
            # в окресности опыт поищем 'без'
            if is_word_near_exp(no_exp, desc_tokens, i):
                # пока запомним
                is_need_exp = 0
            
            # в окресности опыт поищем 'только' или 'обязателен'
            for h in need_exp:
                if is_word_near_exp(h, desc_tokens, i, 20):
                    # пока запомним
                    is_need_exp = default_exp
            
            # опыт работа - считаем что нужен
            for value in desc_tokens[i+1:i+1+2]:
                if 'работа' in value:
                    is_need_exp = default_exp
            
    return is_need_exp
            

In [914]:
from tqdm import tqdm
exp_test = ['Опыт работы с Java от 3-х лет;',
'опыт работы не менее 2-х лет',
'аналогичный опыт работы от года;',
'опыт работы в продажах электро и другого сложного оборудования обязателен.',
'тебя есть опыт преподавания от 6 месяцев;',
'Возможно оформление без опыта работы',
'Опыт работы желателен.',
'опыт работы по аналогичному направлению деятельности не менее 3-х лет;',
'Опыт программирования в 1С 8 не менее 3-х лет;',
'Имеется опыт в IT recruiting (инхаус от 1 года или агентство от 6 месяцев),',
'Возможно без опыта, но большое желание его получить',
'На вакансию рассматриваются только кандидаты с опытом работы в сфере HR',
'Опыт работы <strong>в найме персонала</strong> от 2-х лет Опыт найма сотрудников, которые работают в компании год и более;<',
'Релевантный опыт работы как преимущество;</p> <p>- Приветствуется опыт в сетевых компаниях;',
'отделом технической поддержки (опыт от 1-ого года) ',
'Имеете опыт общения с конечным потребителем софта',
'опыт работы с детьми от 1 года;',
'с художественным академическим образованием и опытом более 5 лет',
'Рассматриваем только с опытом продажи',
'Опыт работы от года;',
'Управленческий опыт работы от 0,5 года (желательно)',
'опыт работы в PR от 2-х лет;',
'Опыт работы с sql, kotlin. Опыт работы на руководящей должность от 1 года',
'Опыт работы в данной сфере обязателен, будет плюсом опыт в химчистке v Опыт работы с sql, kotlin.',
'Имеет опыт работы в интернет-маркетинге от 2-х лет']

# ttttt = df.loc[0:1000, 'description']

# ttttt - токенизировано mod=0

tokens_exp=[tokenize(a,is_lemmatization=True,mod = 0) for a in exp_test]

h = [desc_to_tokens(a,  False, False) for a in tokens_exp]
h1 = [get_experience(u) for u in h]


for i, j in zip(exp_test, h1):
    print(f'{i}\n -> {j}')

Опыт работы с Java от 3-х лет;
 -> 3
опыт работы не менее 2-х лет
 -> 2
аналогичный опыт работы от года;
 -> 1
опыт работы в продажах электро и другого сложного оборудования обязателен.
 -> 1
тебя есть опыт преподавания от 6 месяцев;
 -> 0.5
Возможно оформление без опыта работы
 -> 1
Опыт работы желателен.
 -> 1
опыт работы по аналогичному направлению деятельности не менее 3-х лет;
 -> 3
Опыт программирования в 1С 8 не менее 3-х лет;
 -> 3
Имеется опыт в IT recruiting (инхаус от 1 года или агентство от 6 месяцев),
 -> 1.0
Возможно без опыта, но большое желание его получить
 -> 0
На вакансию рассматриваются только кандидаты с опытом работы в сфере HR
 -> 1
Опыт работы <strong>в найме персонала</strong> от 2-х лет Опыт найма сотрудников, которые работают в компании год и более;<
 -> 2
Релевантный опыт работы как преимущество;</p> <p>- Приветствуется опыт в сетевых компаниях;
 -> 1
отделом технической поддержки (опыт от 1-ого года) 
 -> 1
Имеете опыт общения с конечным потребителем софта


In [915]:
test_df = X_train.loc[0:99,['id','description' ]]
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           100 non-null    int64 
 1   description  100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [916]:
tokens_description_test=[tokenize(a,is_lemmatization=True,mod = 0) for a in tqdm(test_df.description)]
len(tokens_description_test)

100%|██████████| 100/100 [00:05<00:00, 19.74it/s]


100

In [917]:
h_exp_test = [desc_to_tokens(a, False, False) for a in tokens_description_test]
h1_exp_test = [get_experience(u) for u in h_exp_test]
len(h_exp_test),len(h1_exp_test)

(100, 100)

In [918]:
h_eng_test = [desc_to_tokens(a, True) for a in tqdm(tokens_description_test)]
h1_eng_test = [get_english(t) for t in h_eng_test]
len(h_eng_test ),len(h1_eng_test)

100%|██████████| 100/100 [00:04<00:00, 21.08it/s]


(100, 100)

In [919]:
test_df['exp'] = h1_exp_test
test_df['eng'] = h1_eng_test
test_df

,id,description,exp,eng
0,29083,<strong>Обязанности:</strong> <p>​​​​​- Прием ...,1.0,None
1,26052,<p><strong>CityAds Media</strong> – один из ли...,2.0,B
2,24055,<p>В перспективный международный Fintech старт...,2.0,None
3,4408,<p><strong>Обязанности:</strong></p><ul><li>Об...,0.0,None
4,16856,<p><strong>Компания MALL DECOR</strong> - лиде...,0.0,None
...,...,...,...,...
95,17956,<strong>Обязанности:</strong> <ul> <li>Разрабо...,0.0,None
96,16629,<p><strong>Темп в поиске харизматичных зумеров...,1.0,None
97,22786,<p><strong>Обязанности:</strong></p> <ul> <li>...,1.0,None
98,35738,<p><strong>Обязанности:</strong></p> <ul> <li>...,1.0,None


In [920]:
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0.pkl", 'rb') as file:
        tokens_description_lemm_mod0 = pickle.load(file)
else:
    tokens_description_lemm_mod0=[tokenize(a,is_lemmatization=True,mod = 0) for a in tqdm(X_train.description)]
    with open('tokens_description_plus_lemment_mod0.pkl', 'wb') as file:
        pickle.dump(tokens_description_lemm_mod0, file)

In [921]:
# h_exp_full = [desc_to_tokens(a, False, False) for a in tokens_description_lemm_mod0]
# h1_exp_full = [get_experience(u) for u in h_exp_full]
# len(h_exp_full),len(h1_exp_full)

In [922]:
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_eng.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_eng.pkl", 'rb') as file:
        h_eng_full = pickle.load(file)
else:
    h_eng_full = [desc_to_tokens(a, True) for a in tqdm(tokens_description_lemm_mod0)]
    with open('tokens_description_plus_lemment_mod0_eng.pkl', 'wb') as file:
        pickle.dump(h_eng_full, file)

In [923]:
# h_eng_full = [desc_to_tokens(a, True) for a in tokens_description_lemm_mod0]
# h1_eng_full = [get_english(t) for t in h_eng_full]
# len(h_eng_full ),len(h1_eng_full)

In [924]:
if(os.path.exists("/kaggle/input/nsu-jobs/Exp_Eng.csv")):
    Eng_Exp = pd.read_csv("/kaggle/input/nsu-jobs/Exp_Eng.csv")
    X_train=X_train.merge(Eng_Exp ,how='left', on='id')
    X_train.eng = X_train.eng.fillna('')
else:
    h_exp_full = [desc_to_tokens(a, False, False) for a in tokens_description_lemm_mod0]
    h1_exp_full = [get_experience(u) for u in h_exp_full]
    
    h1_eng_full = [get_english(t) for t in h_eng_full]
    
    X_train['exp'] = h1_exp_full
    X_train['eng'] = h1_eng_full
    X_train[['id','exp','eng']].to_csv('Exp_Eng.csv', index=False)
    X_train.eng = X_train.eng.fillna('')

In [925]:
# X_train['exp'] = h1_exp_full
# X_train['eng'] = h1_eng_full
# X_train[['id','exp','eng']].to_csv('Exp_Eng.csv', index=False)

In [926]:
# Eng_Exp = pd.read_csv("/kaggle/input/nsu-jobs/Exp_Eng.csv")
# X_train=X_train.merge(Eng_Exp ,how='left', on='id')
# X_train.eng = X_train.eng.fillna('')

In [927]:
X_train[['id','exp','eng']]

,id,exp,eng
0,29083,1.0,
1,26052,2.0,B
2,24055,2.0,
3,4408,0.0,
4,16856,0.0,
...,...,...,...
27929,16850,3.0,B
27930,6265,0.0,
27931,11284,1.0,
27932,860,0.0,B


In [928]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        27934 non-null  int64              
 1   name                      27934 non-null  object             
 2   has_test                  27934 non-null  bool               
 3   response_letter_required  27934 non-null  bool               
 4   salary_from               23902 non-null  float64            
 5   salary_currency           27934 non-null  object             
 6   salary_gross              27934 non-null  bool               
 7   published_at              27934 non-null  datetime64[ns, UTC]
 8   created_at                27934 non-null  datetime64[ns, UTC]
 9   employer_name             27934 non-null  object             
 10  description               27934 non-null  object             
 11  area_id        

In [929]:
def is_bonus(tokens_lemm: list):
    bonus_words = ['бонус', 'премия', 'процент']
    return any(map(lambda bonus_word: bonus_word in tokens_lemm, bonus_words))

In [930]:
desc_is_bonus=[is_bonus(t) for t in tokens_description_lemm_mod0]
X_train['is_bonus']=desc_is_bonus
len(desc_is_bonus)

27934

In [931]:
X_train[X_train.loc[:, X_train.dtypes == object].columns] = X_train.loc[:, X_train.dtypes == object].astype('category')

# Parse Salary from description

In [932]:
def find_salary(tokens: list):
    
    def is_real_zp(zp: int):
        
        salary_theoretical_max = 1000000
        salary_theoretical_min = 10000
        
        return zp <= salary_theoretical_max and zp >= salary_theoretical_min
            
    stopwords_salary = ['партнер', 'клиент', 'партнёр', 'ученик', 'компания', 'сотрудник']    
        
    regular_digit = r'\b[1-9]\d{1,2}(к|т|тыс){1}\b'   # 50к 500к 50тыс
    
    # слово тыс слово 50 100 слово слово тыс -> слово тыс слово 50000 100000 слово слово тыс
    for position in range(len(tokens)):
        if tokens[position] == 'тыс' or tokens[position] == 'к' or tokens[position] == 'т':
            how_many = 0
            for i in range(5):
                if position-i-1 < 0:
                    break
                if tokens[position-i-1].isdigit():
                    how_many += 1
                    tokens[position-i-1] += "000"
                    if how_many == 2:
                        break
    
    tokens = list(filter(lambda t: not (t == 'тыс' or t == 'к' or t == 'т') , tokens))      
    
    # 50к -> 50000   50тыс -> 50000   50т -> 50000  000к -> ничего
    for i in range(len(tokens)):
        if re.match(regular_digit, tokens[i]):
            tokens[i] = ''.join(filter(str.isdigit, tokens[i])) + "000"
            
    # 50 000 -> 50000    50 000р -> 50000    50 000руб -> 50000   50 000к -> 50000 
    for i in range(len(tokens)):
        if re.search(r'\b000(?:р|руб|к)*\b', tokens[i]):
            if i-1 >= 0 and i-1 < len(tokens) and tokens[i-1].isdigit():
                tokens[i-1] = tokens[i-1] + "000"
                tokens[i] = ""
#             if i-2 >= 0 and i-2 < len(tokens) and tokens[i-2].isdigit():
#                 tokens[i-2] = tokens[i-2] + "000"
#                 tokens[i] = ""
#             if i-3 >= 0 and i-3 < len(tokens) and tokens[i-3].isdigit():
#                 tokens[i-3] = tokens[i-3] + "000"
#                 tokens[i] = ""

    tokens = list(filter(bool, tokens)) 

    
    # 50000р -> 50000   50000руб -> 50000 
    for i in range(len(tokens)):
        if re.search(r'\b\d{5,7}(?:р|(?:руб)?)\b', tokens[i]):
            tokens[i] = ''.join(filter(str.isdigit, tokens[i]))
            
    
    min_zp_set = set()
    max_zp_set = set()
    
    # ищем до
    for i in range(len(tokens)):
        if tokens[i].isdigit() and is_real_zp(int(tokens[i])):
            if (i-1 < len(tokens)) and tokens[i-1] == 'до' and (((i+1) == len(tokens)) or (tokens[i+1] not in stopwords_salary)):
                max_zp_set.add(int(tokens[i]))

    # ищем от или не менее
    for i in range(len(tokens)):
        if tokens[i].isdigit() and is_real_zp(int(tokens[i])):
            if (i-1 >= 0) and \
            (((i+1) == len(tokens)) or (tokens[i+1] not in stopwords_salary)) and \
            ((tokens[i-1] == 'от') or ((tokens[i-1] == 'менее') and (i-2 >= 0) and (tokens[i-2] == 'не'))):
                min_zp_set.add(int(tokens[i]))

    max_zp = None
    min_zp = None
    
    # оставляем только максимальный
    max_zp = max(max_zp_set, default=None)
    
    if len(min_zp_set) == 1:
        min_zp = min(min_zp_set, default=None)
    
    # если максимум найден, вывожу его
    if max_zp and min_zp:
        return min_zp, max_zp

    if max_zp:
        return None, max_zp
    
    digits = set()
    
    # оставляю уникальные числа, после которых нет стопслов
    for i in range(len(tokens)):
        if (tokens[i].isdigit() and is_real_zp(int(tokens[i]))) and (((i+1) == len(tokens)) or (tokens[i+1] not in stopwords_salary)):
            digits.add(int(tokens[i]))
            
    digits = list(digits)
    
    # сортировка
    digits.sort(reverse=True)
    
    # если я знаю ОТ min_zp, и если максимум больше него, то возвращаю максимум
    if min_zp:
        if len(digits) >= 1 and digits[0] > min_zp:
            max_zp = digits[0]
            return min_zp, max_zp
        else:
             # если максимума нет
            return min_zp, None
        
    # если я не знаю ОТ min_zp, то ищу два максимума
    
    # беру два самых больших
    if len(digits) >= 1:
        max_zp = digits[0]
        
    if len(digits) >= 2:
        min_zp = digits[1]  
        
    return min_zp, max_zp

In [933]:
test_df = X_train[['id','description', 'is_bonus', 'salary_from', 'salary_to']][1000:1200]
descs_tokens = [tokenize(d,is_lemmatization=True,mod = 1) for d in test_df['description']]
descs_tokens = [desc_to_tokens(d) for d in descs_tokens]
oklads = [find_salary(desc_tokens) for desc_tokens in descs_tokens]

In [934]:
test_df['oklad'] = oklads
test_df

,id,description,is_bonus,salary_from,salary_to,oklad
1000,1414,"<p>Приглашаем стать членом нашей команды, в От...",False,35000.0,35000.0,"(None, None)"
1001,23209,<p><strong>Обязанности:</strong></p> <ul> <li>...,False,14000.0,28000.0,"(None, None)"
1002,25930,<p><strong>Обязанности:</strong></p> <p>- Выпо...,False,65000.0,70000.0,"(None, None)"
1003,20024,<p><strong>Должностные обязанности:</strong></...,False,45000.0,50000.0,"(None, None)"
1004,36433,<strong>Обязанности:</strong> <p>- -Выполнение...,False,NaN,70000.0,"(None, None)"
...,...,...,...,...,...,...
1195,35973,<p><strong>Digital-агентство Web Generation Gr...,False,NaN,40000.0,"(30000, None)"
1196,13590,<p>В группу хроматоргафии отделения комбинатор...,False,45000.0,70000.0,"(None, None)"
1197,20632,<strong>Обязанности:</strong> <ul> <li>Опыт ра...,False,60000.0,100000.0,"(None, None)"
1198,9459,"<p>В компанию, которая специализируется на ИТ ...",False,80000.0,120000.0,"(None, None)"


In [935]:
salary_min_desc = []
salary_max_desc = []
for ind, oklad in enumerate(oklads):
    salary_from = test_df.salary_from[1000+ind]
    if(oklad[1]==salary_from):
        salary_min_desc.append(oklad[1])
        salary_max_desc.append(None)
    else:
        salary_min_desc.append(oklad[0])
        salary_max_desc.append(oklad[1])

In [936]:
test_df['salary_min_desc']=salary_min_desc
test_df['salary_max_desc']=salary_max_desc

In [937]:
test_df.loc[((test_df.salary_from>test_df.salary_max_desc) | (abs(test_df['salary_min_desc']-test_df.salary_from)>test_df.salary_from*0.1)),['salary_min_desc', 'salary_max_desc']] = None 
test_df.loc[1190:1200]

,id,description,is_bonus,salary_from,salary_to,oklad,salary_min_desc,salary_max_desc
1190,20055,<strong>Обязанности:</strong> <ul> <li> <p>Экс...,False,85000.0,100000.0,"(None, None)",NaN,NaN
1191,4907,<p>Yva.ai – компания спин-офф от ABBYY Давида ...,True,40000.0,50000.0,"(10000, 40000)",40000.0,NaN
1192,18236,"<p>М13 - компания, которая занимается разработ...",False,130000.0,170000.0,"(None, None)",NaN,NaN
1193,24621,<p><strong>Обязанности</strong>:</p> <ul> <li>...,False,65342.0,84945.0,"(None, None)",NaN,NaN
1194,367,<p> Группа компаний «Велунд Сталь» зарекомендо...,False,85000.0,240000.0,"(None, 30000)",NaN,NaN
1195,35973,<p><strong>Digital-агентство Web Generation Gr...,False,NaN,40000.0,"(30000, None)",30000.0,NaN
1196,13590,<p>В группу хроматоргафии отделения комбинатор...,False,45000.0,70000.0,"(None, None)",NaN,NaN
1197,20632,<strong>Обязанности:</strong> <ul> <li>Опыт ра...,False,60000.0,100000.0,"(None, None)",NaN,NaN
1198,9459,"<p>В компанию, которая специализируется на ИТ ...",False,80000.0,120000.0,"(None, None)",NaN,NaN
1199,36140,<p>Контроль размещения товара на складе</p> <p...,False,30000.0,50000.0,"(None, None)",NaN,NaN


In [938]:
import pickle
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod1.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod1.pkl", 'rb') as file:
        tokens_description_lemm_mod1 = pickle.load(file)
else:
    tokens_description_lemm_mod1 = [tokenize(d,is_lemmatization=True,mod = 1) for d in tqdm(X_train['description'])]
    with open('tokens_description_plus_lemment_mod1.pkl', 'wb') as file:
        pickle.dump(tokens_description_lemm_mod1, file)

In [939]:
descs_tokens_rus = list(map(lambda desc: list(filter(lambda token: is_rus(token), desc)), tqdm(tokens_description_lemm_mod1)))
len(descs_tokens_rus)

100%|██████████| 27934/27934 [00:37<00:00, 744.98it/s]


27934

In [940]:
descs_tokens_full = [desc_to_tokens(d) for d in tqdm(descs_tokens_rus)]

100%|██████████| 27934/27934 [00:05<00:00, 5032.22it/s]


In [941]:
oklads_full = [find_salary(descs_token) for descs_token in tqdm(descs_tokens_full)]

100%|██████████| 27934/27934 [00:16<00:00, 1715.63it/s]


In [942]:
salary_min_desc_full = []
salary_max_desc_full = []
for ind, oklad in enumerate(oklads_full):
    salary_from = X_train.salary_from[ind]
    if(oklad[1]==salary_from):
        salary_min_desc_full.append(oklad[1])
        salary_max_desc_full.append(None)
    else:
        salary_min_desc_full.append(oklad[0])
        salary_max_desc_full.append(oklad[1])

In [943]:
X_train['salary_min_desc']=salary_min_desc_full
X_train['salary_max_desc']=salary_max_desc_full
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        27934 non-null  int64              
 1   name                      27934 non-null  category           
 2   has_test                  27934 non-null  bool               
 3   response_letter_required  27934 non-null  bool               
 4   salary_from               23902 non-null  float64            
 5   salary_currency           27934 non-null  category           
 6   salary_gross              27934 non-null  bool               
 7   published_at              27934 non-null  datetime64[ns, UTC]
 8   created_at                27934 non-null  datetime64[ns, UTC]
 9   employer_name             27934 non-null  category           
 10  description               27934 non-null  category           
 11  area_id        

In [944]:
X_train.loc[((X_train.salary_from>X_train.salary_max_desc) | (abs(X_train['salary_min_desc']-X_train.salary_from)>X_train.salary_from*0.1)),['salary_min_desc', 'salary_max_desc']] = None 

In [945]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        27934 non-null  int64              
 1   name                      27934 non-null  category           
 2   has_test                  27934 non-null  bool               
 3   response_letter_required  27934 non-null  bool               
 4   salary_from               23902 non-null  float64            
 5   salary_currency           27934 non-null  category           
 6   salary_gross              27934 non-null  bool               
 7   published_at              27934 non-null  datetime64[ns, UTC]
 8   created_at                27934 non-null  datetime64[ns, UTC]
 9   employer_name             27934 non-null  category           
 10  description               27934 non-null  category           
 11  area_id        

# Считаем средние зп, по ним заполняем недостающие.

In [946]:
agg_prof1_emp = X_train.groupby(['start_name1','employer_name'],sort=False).agg(
    salary_mean_prof1_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_prof2_emp = X_train.groupby(['start_name2','employer_name'],sort=False).agg(
    salary_mean_prof2_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_prof3_emp = X_train.groupby(['start_name3','employer_name'],sort=False).agg(
    salary_mean_prof3_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_emp_area = X_train.groupby(['employer_name','area_id'],sort=False).agg(
    salary_mean_emp_area = ('salary_from', lambda x: np.mean(x)),
)
agg_prof1_area = X_train.groupby(['start_name1','area_id'],sort=False).agg(
    salary_mean_prof1_area = ('salary_from', lambda x: np.mean(x)),
)
agg_emp = X_train.groupby(['employer_name',],sort=False).agg(
    salary_mean_emp = ('salary_from', lambda x: np.mean(x)),
)
agg_prof = X_train.groupby(['start_name1',],sort=False).agg(
    salary_mean_prof = ('salary_from', lambda x: np.mean(x)),
)
agg_area = X_train.groupby(['area_id',],sort=False).agg(
    salary_mean_area = ('salary_from', lambda x: np.mean(x)),
)
agg_all = X_train.salary_from.mean()

agg_prof12 = X_train.groupby(['start_name1','start_name2',],sort=False).agg(
    salary_mean_prof12 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof13 = X_train.groupby(['start_name1','start_name3',],sort=False).agg(
    salary_mean_prof13 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof23 = X_train.groupby(['start_name2','start_name3',],sort=False).agg(
    salary_mean_prof23 = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_emp = X_train.groupby(['optimum_title','employer_name'],sort=False).agg(
    salary_mean_opt_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_area = X_train.groupby(['optimum_title','area_id'],sort=False).agg(
    salary_mean_opt_area = ('salary_from', lambda x: np.mean(x)),
)

agg_opt = X_train.groupby(['optimum_title'],sort=False).agg(
    salary_mean_opt = ('salary_from', lambda x: np.mean(x)),
)
agg_opt_prof1 = X_train.groupby(['optimum_title', 'start_name1'],sort=False).agg(
    salary_mean_opt_prof1 = ('salary_from', lambda x: np.mean(x)),
)

In [947]:
agg_prof12 = X_train.groupby(['start_name1','start_name2',],sort=False).agg(
    salary_mean_prof12 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof13 = X_train.groupby(['start_name1','start_name3',],sort=False).agg(
    salary_mean_prof13 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof23 = X_train.groupby(['start_name2','start_name3',],sort=False).agg(
    salary_mean_prof23 = ('salary_from', lambda x: np.mean(x)),
)

In [948]:
agg_opt_emp = X_train.groupby(['optimum_title','employer_name'],sort=False).agg(
    salary_mean_opt_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_area = X_train.groupby(['optimum_title','area_id'],sort=False).agg(
    salary_mean_opt_area = ('salary_from', lambda x: np.mean(x)),
)

agg_opt = X_train.groupby(['optimum_title'],sort=False).agg(
    salary_mean_opt = ('salary_from', lambda x: np.mean(x)),
)
agg_opt_prof1 = X_train.groupby(['optimum_title', 'start_name1'],sort=False).agg(
    salary_mean_opt_prof1 = ('salary_from', lambda x: np.mean(x)),
)

In [949]:
agg_opt_emp.reset_index(inplace=True)
agg_opt_area.reset_index(inplace=True)
agg_opt.reset_index(inplace=True)
agg_opt_prof1.reset_index(inplace=True)

agg_prof1_emp.reset_index(inplace=True)
agg_prof2_emp.reset_index(inplace=True)
agg_prof3_emp.reset_index(inplace=True)
agg_emp_area.reset_index(inplace=True)
agg_prof1_area.reset_index(inplace=True)
agg_emp.reset_index(inplace=True)
agg_prof.reset_index(inplace=True)
agg_area.reset_index(inplace=True)
agg_prof12.reset_index(inplace=True)
agg_prof13.reset_index(inplace=True)
agg_prof23.reset_index(inplace=True)

In [950]:
agg_opt_emp=agg_opt_emp.dropna()
agg_opt_area=agg_opt_area.dropna()
agg_opt=agg_opt.dropna()
agg_opt_prof1=agg_opt_prof1.dropna()

agg_prof1_emp = agg_prof1_emp.dropna()
agg_prof2_emp = agg_prof2_emp.dropna()
agg_prof3_emp = agg_prof3_emp.dropna()
agg_emp_area = agg_emp_area.dropna()
agg_prof1_area = agg_prof1_area.dropna()
agg_emp = agg_emp.dropna()
agg_prof = agg_prof.dropna()
agg_area = agg_area.dropna()
agg_prof12 = agg_prof12.dropna()
agg_prof13 = agg_prof13.dropna()
agg_prof23 = agg_prof23.dropna()

In [951]:
len(agg_prof1_emp), len(agg_emp_area), len(agg_prof1_area), len(agg_emp),len(agg_prof),len(agg_area),len(agg_prof12), len(agg_prof13), len(agg_prof23),

(16774, 13112, 2200, 11980, 925, 150, 5025, 6440, 6632)

In [952]:
X_train2 = X_train.copy()
without_salary = X_train2[X_train2.salary_from.isnull()]

In [953]:
without_salary = without_salary.merge(agg_opt_emp, on=['optimum_title','employer_name'], how='left')
without_salary = without_salary.merge(agg_opt_area, on=['optimum_title','area_id'], how='left')
without_salary = without_salary.merge(agg_opt, on=['optimum_title'], how='left')
without_salary = without_salary.merge(agg_opt_prof1, on=['optimum_title', 'start_name1'], how='left')

without_salary = without_salary.merge(agg_prof12, on=['start_name1','start_name2'], how='left')
without_salary = without_salary.merge(agg_prof13, on=['start_name1','start_name3'], how='left')
without_salary= without_salary.merge(agg_prof23, on=['start_name2','start_name3'], how='left')

without_salary = without_salary.merge(agg_prof1_emp, on=['start_name1','employer_name'], how='left')
without_salary = without_salary.merge(agg_prof2_emp, on=['start_name2','employer_name'], how='left')
without_salary = without_salary.merge(agg_prof3_emp, on=['start_name3','employer_name'], how='left')

without_salary = without_salary.merge(agg_emp_area, on=['employer_name','area_id'], how='left')
without_salary = without_salary.merge(agg_prof1_area, on=['start_name1','area_id'], how='left')
without_salary = without_salary.merge(agg_emp, on=['employer_name',], how='left')
without_salary = without_salary.merge(agg_prof, on=['start_name1',], how='left')
without_salary = without_salary.merge(agg_area, on=['area_id',], how='left')

In [954]:
without_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        4032 non-null   int64              
 1   name                      4032 non-null   category           
 2   has_test                  4032 non-null   bool               
 3   response_letter_required  4032 non-null   bool               
 4   salary_from               0 non-null      float64            
 5   salary_currency           4032 non-null   category           
 6   salary_gross              4032 non-null   bool               
 7   published_at              4032 non-null   datetime64[ns, UTC]
 8   created_at                4032 non-null   datetime64[ns, UTC]
 9   employer_name             4032 non-null   category           
 10  description               4032 non-null   category           
 11  area_id          

In [955]:
# without_salary[['salary_from','salary_to','salary_mean_opt_emp','salary_mean_opt_prof1','salary_mean_opt_area','salary_mean_opt','salary_mean_prof12', 'salary_mean_prof13', 'salary_mean_prof23','salary_mean_prof1_emp','salary_mean_prof2_emp', 'salary_mean_prof3_emp','salary_mean_emp_area','salary_mean_prof1_area','salary_mean_emp', 'salary_mean_prof','salary_mean_area']].sample(12)

In [956]:
from tqdm import tqdm
median_emp=[]
median_prof = []
for i in tqdm(range(len(without_salary))):
    row_values_emp = without_salary.loc[i, ['salary_mean_prof1_emp','salary_mean_prof2_emp', 'salary_mean_prof3_emp','salary_mean_opt_emp']]
    row_values_prof = without_salary.loc[i, ['salary_mean_opt_prof1','salary_mean_opt_area', 'salary_mean_opt','salary_mean_prof12','salary_mean_prof13','salary_mean_prof23','salary_mean_emp_area','salary_mean_prof1_area','salary_mean_emp','salary_mean_prof']]
    median_emp.append(row_values_emp.mean())
    median_prof.append(row_values_prof.mean())

100%|██████████| 4032/4032 [00:09<00:00, 422.90it/s]


In [957]:
without_salary['median_emp'] = median_emp
without_salary['median_prof'] = median_prof

In [958]:
without_salary.loc[(without_salary.salary_from.isnull()),'salary_from'] = without_salary.loc[(without_salary.salary_from.isnull())].median_emp
without_salary.loc[(without_salary.salary_from.isnull()),'salary_from'] = without_salary.loc[(without_salary.salary_from.isnull())].median_prof
without_salary.loc[(without_salary.salary_from.isnull()),'salary_from'] = without_salary.loc[(without_salary.salary_from.isnull())].salary_mean_area

In [959]:
without_salary = without_salary.rename (columns= {'salary_from': 'salary_from_without'})

In [960]:
without_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        4032 non-null   int64              
 1   name                      4032 non-null   category           
 2   has_test                  4032 non-null   bool               
 3   response_letter_required  4032 non-null   bool               
 4   salary_from_without       4032 non-null   float64            
 5   salary_currency           4032 non-null   category           
 6   salary_gross              4032 non-null   bool               
 7   published_at              4032 non-null   datetime64[ns, UTC]
 8   created_at                4032 non-null   datetime64[ns, UTC]
 9   employer_name             4032 non-null   category           
 10  description               4032 non-null   category           
 11  area_id          

# Добавляем в трейн

In [961]:
X_train2 = X_train.copy()
X_train2 = X_train2.merge(agg_opt_emp, on=['optimum_title','employer_name'], how='left')
X_train2 = X_train2.merge(agg_opt_area, on=['optimum_title','area_id'], how='left')
X_train2 = X_train2.merge(agg_opt, on=['optimum_title'], how='left')
X_train2 = X_train2.merge(agg_opt_prof1, on=['optimum_title', 'start_name1'], how='left')

X_train2 = X_train2.merge(agg_prof12, on=['start_name1','start_name2'], how='left')
X_train2 = X_train2.merge(agg_prof13, on=['start_name1','start_name3'], how='left')
X_train2 = X_train2.merge(agg_prof23, on=['start_name2','start_name3'], how='left')

X_train2 = X_train2.merge(agg_prof1_emp, on=['start_name1','employer_name'], how='left')
X_train2 = X_train2.merge(agg_prof2_emp, on=['start_name2','employer_name'], how='left')
X_train2 = X_train2.merge(agg_prof3_emp, on=['start_name3','employer_name'], how='left')

X_train2 = X_train2.merge(agg_emp_area, on=['employer_name','area_id'], how='left')
X_train2 = X_train2.merge(agg_prof1_area, on=['start_name1','area_id'], how='left')
X_train2 = X_train2.merge(agg_emp, on=['employer_name',], how='left')
X_train2 = X_train2.merge(agg_prof, on=['start_name1',], how='left')
X_train2 = X_train2.merge(agg_area, on=['area_id',], how='left')

In [962]:
X_train2= X_train2.merge(without_salary[['id','salary_from_without']], on='id', how='left')
X_train2.loc[(X_train2.salary_from.isnull()),'salary_from'] = X_train2.salary_from_without

In [963]:
X_train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        27934 non-null  int64              
 1   name                      27934 non-null  category           
 2   has_test                  27934 non-null  bool               
 3   response_letter_required  27934 non-null  bool               
 4   salary_from               27934 non-null  float64            
 5   salary_currency           27934 non-null  category           
 6   salary_gross              27934 non-null  bool               
 7   published_at              27934 non-null  datetime64[ns, UTC]
 8   created_at                27934 non-null  datetime64[ns, UTC]
 9   employer_name             27934 non-null  category           
 10  description               27934 non-null  category           
 11  area_id        

In [964]:
# X_train2['difff'] = abs(X_train1.salary_from - X_train1.salary_to)
# pd.DataFrame({'from':X_train1.salary_from, 'to': X_train1.salary_to, 'difff': X_train1.difff}).sort_values(by='difff').tail(10)

In [965]:
X_train2[X_train2.salary_from<5000]

,id,name,has_test,response_letter_required,salary_from,salary_currency,salary_gross,published_at,created_at,employer_name,description,area_id,area_name,salary_to,timestamp,start_name1,start_name2,start_name3,optimum_title,exp,eng,is_bonus,salary_min_desc,salary_max_desc,salary_mean_opt_emp,salary_mean_opt_area,salary_mean_opt,salary_mean_opt_prof1,salary_mean_prof12,salary_mean_prof13,salary_mean_prof23,salary_mean_prof1_emp,salary_mean_prof2_emp,salary_mean_prof3_emp,salary_mean_emp_area,salary_mean_prof1_area,salary_mean_emp,salary_mean_prof,salary_mean_area,salary_from_without
96,16629,Преподаватель математики (онлайн),False,False,4500.0,RUR,True,2021-09-01 11:25:03+00:00,2021-09-01 11:25:03+00:00,Онлайн-школа Темп,<p><strong>Темп в поиске харизматичных зумеров...,1,Москва,55000.0,1.630496e+09,преподаватель,математика,онлайн,преподаватель математика,1.0,,False,NaN,NaN,4500.0,31250.000000,30781.250000,30781.250000,30781.250000,33718.750000,28038.461538,4500.0,4500.0,4500.0,4500.0,33835.443038,4500.0,32039.837398,66495.421155,NaN
466,35514,Мастер маникюра и педикюра (м),False,False,60.0,RUR,False,2020-07-12 10:27:10+00:00,2020-07-12 10:27:10+00:00,HEIM beauty studio,<p><strong>В салон красоты HEIM BEAUTY требует...,1,Москва,100000.0,1.594550e+09,мастер,маникюр,педикюр,мастер маникюр,0.0,,False,NaN,NaN,60.0,49235.384615,45003.333333,46474.117647,46474.117647,44451.111111,44451.111111,60.0,60.0,60.0,60.0,57544.855556,60.0,54741.975000,66495.421155,NaN
481,23899,QA engineer (инженер по качеству / тестировщик...,False,False,40.0,RUR,False,2020-04-09 01:55:19+00:00,2020-04-09 01:55:19+00:00,Роадгид,Автомобильные видео-регистраторы Roadgid прода...,2,Санкт-Петербург,55.0,1.586397e+09,qa,engineer,инженер,инженер,0.0,B,False,NaN,NaN,40.0,65223.333333,70845.042641,95560.000000,89585.000000,25020.000000,25020.000000,40.0,40.0,40.0,40.0,126523.478261,40.0,125690.344828,53066.703900,NaN
816,25278,Секретарь,False,False,1000.0,RUR,True,2020-04-27 22:10:59+00:00,2020-04-27 22:10:59+00:00,Дельта +,<strong>Обязанности:</strong> <ul> <li>Делопро...,1,Москва,30000.0,1.588025e+09,секретарь,,,секретарь,0.0,,False,NaN,NaN,1000.0,37136.956522,36069.863014,36530.882353,38466.666667,36774.242424,50805.348185,1000.0,1000.0,1000.0,1000.0,37246.774194,1000.0,37369.473684,66495.421155,NaN
1105,18751,Копирайтер технических текстов в мототематике,False,False,2000.0,RUR,False,2021-11-15 06:58:28+00:00,2021-11-15 06:58:28+00:00,СМ-Моторс,<p><strong>Компания &quot;SM - Motors&quot;( Ф...,1,Москва,80000.0,1.636960e+09,копирайтер,технический,текст,копирайтер,0.0,,False,NaN,NaN,2000.0,41851.073276,40703.423387,40983.968153,14666.666667,4666.666667,2000.000000,2000.0,2000.0,2000.0,2000.0,42241.380952,2000.0,40983.968153,66495.421155,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27242,15034,Investment Analyst/ Investment Associate,False,False,150.0,RUR,False,2021-06-25 12:20:29+00:00,2021-06-25 12:20:29+00:00,Black Lions Capital,<p>Black Lions Capital is a company within Red...,1,Москва,250.0,1.624624e+09,analyst,associate,,analyst,0.0,C,False,NaN,NaN,150.0,93341.666667,106551.923077,65025.000000,150.000000,65025.000000,20075.000000,150.0,150.0,150.0,150.0,58030.000000,150.0,65025.000000,66495.421155,NaN
27618,27236,Классный барбер/ стилист по волосам,False,False,50.0,RUR,True,2020-05-13 21:26:58+00:00,2020-05-13 21:26:58+00:00,Glambar,"<p>Glambar -это новый проект, созданный команд...",1,Москва,120000.0,1.589405e+09,классный,барбер,стилист,барбер,1.0,,False,NaN,NaN,50.0,48010.000000,48010.000000,50.000000,50.000000,50.000000,50.000000,50.0,50.0,50.0,25025.0,50.000000,25025.0,50.000000,66495.421155,NaN
27645,29592,Шеф-повар,False,False,150.0,RUR,False,2020-06-28 18:21:12+00:00,2020-06-28 18:21:12+00:00,Подъяпольская Татьяна Викторовна,<strong>Обязанности:</strong> <ul> <li>Создани...,1,Москва,300.0,1.593368e+09,шеф,повар,,повар,3.0,,False,NaN,N

In [966]:
# X_train2[['salary_from','salary_to','salary_mean_opt_emp','salary_mean_opt_prof1','salary_mean_opt_area','salary_mean_opt','salary_mean_prof12', 'salary_mean_prof13', 'salary_mean_prof23','salary_mean_prof1_emp','salary_mean_prof2_emp', 'salary_mean_prof3_emp','salary_mean_emp_area','salary_mean_prof1_area','salary_mean_emp', 'salary_mean_prof','salary_mean_area']].sample(12)

# Посмотрим крайние ЗП


In [967]:
X_train_low = X_train[X_train.salary_from<=5000]

In [968]:
X_train_norm = X_train[(X_train.salary_from>5000) & (X_train.salary_from<=300000)]

In [969]:
X_train_high = X_train[X_train.salary_from>5000]

In [970]:
len(X_train_low), len(X_train_norm), len(X_train_high)

(135, 23725, 23767)

In [971]:
tes = X_train_low[['id', 'salary_from', 'salary_to', 'salary_min_desc', 'salary_max_desc']]

In [972]:
tes[(~tes.salary_min_desc.isnull())]

,id,salary_from,salary_to,salary_min_desc,salary_max_desc


In [973]:
tes[((~tes.salary_max_desc.isnull()) | (~tes.salary_min_desc.isnull()))]

,id,salary_from,salary_to,salary_min_desc,salary_max_desc
4917,906,5000.0,15000.0,NaN,15000.0
14068,6260,35.0,60.0,NaN,137000.0
16129,26668,5000.0,80000.0,NaN,35000.0
18872,12241,5000.0,200000.0,NaN,200000.0
20184,25928,5000.0,10000.0,NaN,10000.0
20805,34963,1600.0,1800.0,NaN,414000.0
21190,6752,2500.0,2500.0,NaN,50000.0
21938,34492,5000.0,10000.0,NaN,10000.0
21964,16727,5000.0,10000.0,NaN,10000.0
23017,5798,15.0,25.0,NaN,15000.0


# Train

In [974]:
X_train2 = X_train2.drop(columns=['salary_from_without'])
X_train = X_train2.copy()

In [975]:
y_true = X_train.salary_to
X = X_train.drop(columns=["id", "description","salary_currency","salary_to", 'published_at','created_at', 'exp', 'eng', 'is_bonus'])

In [976]:
category_feat = X.loc[:, X.dtypes == 'category'].columns.to_list()
category_feat

['name',
 'employer_name',
 'area_name',
 'start_name1',
 'start_name2',
 'start_name3',
 'optimum_title']

In [977]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27934 entries, 0 to 27933
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   name                      27934 non-null  category
 1   has_test                  27934 non-null  bool    
 2   response_letter_required  27934 non-null  bool    
 3   salary_from               27934 non-null  float64 
 4   salary_gross              27934 non-null  bool    
 5   employer_name             27934 non-null  category
 6   area_id                   27934 non-null  int64   
 7   area_name                 27934 non-null  category
 8   timestamp                 27934 non-null  float64 
 9   start_name1               27934 non-null  category
 10  start_name2               27934 non-null  category
 11  start_name3               27934 non-null  category
 12  optimum_title             27934 non-null  category
 13  salary_min_desc           4100 non-null   floa

In [978]:
X_tr, X_val, y_tr, y_val = train_test_split(X, y_true, test_size=0.2, random_state=912, shuffle=True)


In [979]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor(cat_features=category_feat, iterations=600, learning_rate=0.1, depth=10,score_function= 'L2', random_state=912, boosting_type = 'Plain',has_time=True,max_bin=254, random_strength=1)

In [980]:
cat.fit(X_tr,y_tr)

0:	learn: 98542.7216286	total: 88.9ms	remaining: 53.3s
1:	learn: 93833.8474664	total: 178ms	remaining: 53.3s
2:	learn: 89711.3049513	total: 263ms	remaining: 52.4s
3:	learn: 86185.8888835	total: 346ms	remaining: 51.5s
4:	learn: 82975.7072415	total: 429ms	remaining: 51.1s
5:	learn: 80190.1219058	total: 509ms	remaining: 50.4s
6:	learn: 77732.0717105	total: 593ms	remaining: 50.3s
7:	learn: 75447.0462176	total: 677ms	remaining: 50.1s
8:	learn: 73547.0164284	total: 756ms	remaining: 49.7s
9:	learn: 71774.0221459	total: 835ms	remaining: 49.3s
10:	learn: 70291.0368557	total: 913ms	remaining: 48.9s
11:	learn: 68927.6135481	total: 993ms	remaining: 48.7s
12:	learn: 67663.0038084	total: 1.07s	remaining: 48.5s
13:	learn: 66550.7751027	total: 1.15s	remaining: 48.2s
14:	learn: 65573.8176374	total: 1.23s	remaining: 48.1s
15:	learn: 64613.7053109	total: 1.32s	remaining: 48.1s
16:	learn: 63809.5332563	total: 1.4s	remaining: 48.1s
17:	learn: 63066.5194150	total: 1.49s	remaining: 48.3s
18:	learn: 62342.844

In [981]:

pd.DataFrame({'featchure':X_tr.columns, 'imp': cat.feature_importances_})

,featchure,imp
0,name,0.543096
1,has_test,0.091245
2,response_letter_required,0.932176
3,salary_from,20.769119
4,salary_gross,0.796397
5,employer_name,3.740234
6,area_id,0.322565
7,area_name,1.298008
8,timestamp,9.324886
9,start_name1,7.272916


In [982]:
y_pred_val = cat.predict(X_val)

In [983]:
import numpy as np

def sym_mean_absolute_percentage_error(actual, predicted):
    return 200*np.mean(np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted)))


In [984]:
sym_mean_absolute_percentage_error(y_val,y_pred_val)

22.76694013789546

In [985]:
qw = pd.DataFrame({ 'X_from': X_val.salary_from,'y_true':y_val, 'y_pred': y_pred_val,'salary_max_desc':X_val.salary_max_desc, 'y_pred_copy': y_pred_val  })

In [986]:
qw.loc[~qw.salary_max_desc.isnull(),'y_pred'] = qw.loc[~qw.salary_max_desc.isnull()].salary_max_desc
qw.loc[~qw.salary_max_desc.isnull()]

,X_from,y_true,y_pred,salary_max_desc,y_pred_copy
5739,37000.000000,45000.0,45000.0,45000.0,41944.133060
16333,30000.000000,35000.0,35000.0,35000.0,41849.712796
25790,53953.435549,20000.0,10000.0,10000.0,124772.575704
21308,50000.000000,70000.0,70000.0,70000.0,95381.951668
2632,54629.304029,80000.0,100000.0,100000.0,85708.892547
...,...,...,...,...,...
11316,38333.333333,40000.0,40000.0,40000.0,39678.044906
10596,60000.000000,100000.0,100000.0,100000.0,141247.666687
27094,50000.000000,110000.0,110000.0,110000.0,111447.224782
1782,34500.000000,43000.0,43000.0,43000.0,46833.540733


In [987]:
y_pred_val = qw.y_pred

In [988]:
smape = sym_mean_absolute_percentage_error(y_val,y_pred_val)
print("SMAPE:", smape)

SMAPE: 22.081485768618688


In [989]:
qw['diff'] = abs(qw.y_true-qw.y_pred)

In [990]:
qw[((~qw.salary_max_desc.isnull()))].sort_values(by = 'diff').tail(20)

,X_from,y_true,y_pred,salary_max_desc,y_pred_copy,diff
20004,220000.000000,250000.0,400000.0,400000.0,341374.614708,150000.0
9263,40000.000000,200000.0,352185.0,352185.0,181360.560545,152185.0
3134,100000.000000,300000.0,130000.0,130000.0,194223.832697,170000.0
5981,90000.000000,300000.0,100000.0,100000.0,206046.386948,200000.0
27791,75000.000000,150000.0,350000.0,350000.0,188334.164200,200000.0
25259,162190.404754,250000.0,40000.0,40000.0,304853.695255,210000.0
26549,171889.527972,250000.0,25000.0,25000.0,184231.544355,225000.0
26625,100000.000000,400000.0,150000.0,150000.0,241584.495942,250000.0
22859,32000.000000,300000.0,39000.0,39000.0,93760.389253,261000.0
4487,80000.000000,120000.0,400000.0,400000.0,127316.890880,280000.0


In [991]:
qw.loc[((~qw.salary_max_desc.isnull()) &(qw.y_pred>qw.X_from*2)  ), 'y_pred'] = qw.loc[((~qw.salary_max_desc.isnull()) &(qw.y_pred>qw.X_from*2))].y_pred_copy

In [992]:
qw['diff'] = abs(qw.y_true-qw.y_pred)

In [993]:
qw[((~qw.salary_max_desc.isnull()) )].sort_values(by = 'diff').tail(20)

,X_from,y_true,y_pred,salary_max_desc,y_pred_copy,diff
4253,53423.571870,150000.0,4.000000e+04,40000.0,1.126820e+05,1.100000e+05
7236,58485.882044,150000.0,3.900000e+04,39000.0,7.417586e+04,1.110000e+05
16156,120000.000000,260000.0,1.400000e+05,140000.0,1.757998e+05,1.200000e+05
155,120000.000000,150000.0,2.829312e+05,300000.0,2.829312e+05,1.329312e+05
20004,220000.000000,250000.0,4.000000e+05,400000.0,3.413746e+05,1.500000e+05
11393,263333.333333,400000.0,2.500000e+05,250000.0,3.315149e+05,1.500000e+05
13342,124855.810811,250000.0,1.000000e+05,100000.0,1.957267e+05,1.500000e+05
11522,48000.000000,150000.0,3.036256e+05,150000.0,3.036256e+05,1.536256e+05
3134,100000.000000,300000.0,1.300000e+05,130000.0,1.942238e+05,1.700000e+05
25733,79375.000000,150000.0,3.331544e+05,500000.0,3.331544e+05,1.831544e+05


In [994]:
sym_mean_absolute_percentage_error(qw[((~qw.salary_max_desc.isnull()))].y_true, qw[((~qw.salary_max_desc.isnull()))].y_pred)

13.521420821983588

In [995]:
qw.loc[qw.X_from<150,'y_pred']  = qw.loc[qw.X_from<=150].X_from*1.2
y_pred_val=qw.y_pred

In [996]:
smape = sym_mean_absolute_percentage_error(y_val,y_pred_val)
print("SMAPE:", smape)

SMAPE: 21.831506676481133


# Train all + predict Test

In [997]:
y_true = X_train.salary_to
X = X_train.drop(columns=["id", "description","salary_currency","salary_to", 'published_at','created_at' ])

In [998]:
category_feat = X.loc[:, X.dtypes == 'category'].columns.to_list()
category_feat

['name',
 'employer_name',
 'area_name',
 'start_name1',
 'start_name2',
 'start_name3',
 'optimum_title',
 'eng']

In [999]:
cat = CatBoostRegressor(cat_features=category_feat, iterations=700, learning_rate=0.1, depth=10,score_function= 'L2', random_state=912, boosting_type = 'Plain',has_time=False,max_bin=254, random_strength=1)


In [1000]:
cat.fit(X, y_true)

0:	learn: 100135.6312883	total: 137ms	remaining: 1m 35s
1:	learn: 95525.3097815	total: 257ms	remaining: 1m 29s
2:	learn: 91653.0818792	total: 375ms	remaining: 1m 27s
3:	learn: 88269.2275434	total: 511ms	remaining: 1m 28s
4:	learn: 85350.7190670	total: 631ms	remaining: 1m 27s
5:	learn: 82696.6803922	total: 755ms	remaining: 1m 27s
6:	learn: 80340.4997057	total: 880ms	remaining: 1m 27s
7:	learn: 78403.0994359	total: 1s	remaining: 1m 26s
8:	learn: 76604.0706370	total: 1.13s	remaining: 1m 26s
9:	learn: 75007.8919898	total: 1.24s	remaining: 1m 25s
10:	learn: 73519.3880131	total: 1.36s	remaining: 1m 25s
11:	learn: 72173.7806728	total: 1.48s	remaining: 1m 25s
12:	learn: 71026.1973592	total: 1.58s	remaining: 1m 23s
13:	learn: 70060.1134438	total: 1.72s	remaining: 1m 24s
14:	learn: 69080.2830967	total: 1.84s	remaining: 1m 23s
15:	learn: 68354.3657019	total: 1.96s	remaining: 1m 23s
16:	learn: 67635.8348053	total: 2.08s	remaining: 1m 23s
17:	learn: 66854.4685543	total: 2.2s	remaining: 1m 23s
18:	l

# Test

In [1001]:
X_test = pd.read_csv('/kaggle/input/nsu-jobs/nsu-bda-2023-jobs/X_test.csv')
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        9312 non-null   int64  
 1   name                      9312 non-null   object 
 2   has_test                  9312 non-null   bool   
 3   response_letter_required  9312 non-null   bool   
 4   salary_from               7924 non-null   float64
 5   salary_currency           9312 non-null   object 
 6   salary_gross              9263 non-null   object 
 7   published_at              9312 non-null   object 
 8   created_at                9312 non-null   object 
 9   employer_name             9312 non-null   object 
 10  description               9312 non-null   object 
 11  area_id                   9312 non-null   int64  
 12  area_name                 9312 non-null   object 
dtypes: bool(2), float64(1), int64(2), object(8)
memory usage: 818.6

In [1002]:
X_test.description = X_test.description.fillna('')
X_test.published_at = pd.to_datetime(X_test.published_at, errors='raise')
X_test.created_at = pd.to_datetime(X_test.created_at, errors='raise')
X_test['timestamp'] = (X_test.published_at.astype('int64') / 10**9)
X_test.salary_gross=X_test.salary_gross.astype('bool')

In [1003]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        9312 non-null   int64              
 1   name                      9312 non-null   object             
 2   has_test                  9312 non-null   bool               
 3   response_letter_required  9312 non-null   bool               
 4   salary_from               7924 non-null   float64            
 5   salary_currency           9312 non-null   object             
 6   salary_gross              9312 non-null   bool               
 7   published_at              9312 non-null   datetime64[ns, UTC]
 8   created_at                9312 non-null   datetime64[ns, UTC]
 9   employer_name             9312 non-null   object             
 10  description               9312 non-null   object             
 11  area_id          

In [1004]:
from tqdm import tqdm
names1_test = []
names2_test = []
names3_test = []
for t in tqdm(X_test.name):
    title = preprocess_title(t)
    if(len(title)>=3):
        names1_test.append(title[0])
        names2_test.append(title[1])
        names3_test.append(title[2])
        continue
    elif(len(title)>=2):
        names1_test.append(title[0])
        names2_test.append(title[1])
        names3_test.append('')
        continue
    elif(len(title)>=1):
        names1_test.append(title[0])
        names2_test.append('')
        names3_test.append('')
    else:
        names1_test.append('')
        names2_test.append('')
        names3_test.append('')

100%|██████████| 9312/9312 [00:20<00:00, 464.17it/s]


In [1005]:
X_test['start_name1'] = names1_test
X_test['start_name2'] = names2_test
X_test['start_name3'] = names3_test

In [1006]:
title_counts_test = Counter(X_test['name'].apply(lambda x: " ".join(preprocess_title(x))).values)
del title_counts_test['']
title_counts_test.most_common(10)

[('менеджер продажа', 110),
 ('менеджер работа клиент', 70),
 ('юрист', 62),
 ('frontend разработчик', 56),
 ('помощник юрист', 54),
 ('уборщица уборщик', 49),
 ('копирайтер', 48),
 ('аналитик', 44),
 ('продавец консультант', 40),
 ('менеджер закупка', 39)]

In [1007]:
optimum_titless_test = []
optimum_countss_test = []

for title in tqdm(X_test.name):
    optimum_title = optimize_title(title, title_counts=title_counts_test)
    optimum_count = title_counts[optimum_title]
    optimum_titless_test.append(optimum_title)
    optimum_countss_test.append(optimum_count)

100%|██████████| 9312/9312 [00:20<00:00, 456.14it/s]


In [1008]:
X_test['optimum_title'] = optimum_titless_test

In [1009]:
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_test.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_test.pkl", 'rb') as file:
        tokens_description_lemm_mod0_test = pickle.load(file)
else:
    tokens_description_lemm_mod0_test=[tokenize(a,is_lemmatization=True,mod = 0) for a in tqdm(X_test.description)]
    with open('tokens_description_plus_lemment_mod0_test.pkl', 'wb') as file:
        pickle.dump(tokens_description_lemm_mod0_test, file)

In [1010]:
# h_exp_full_test = [desc_to_tokens(a, False, False) for a in tokens_description_lemm_mod0_test]
# h1_exp_full_test = [get_experience(u) for u in h_exp_full_test]
# len(h_exp_full_test),len(h1_exp_full_test)

In [1011]:
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_eng_test.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod0_eng_test.pkl", 'rb') as file:
        h_eng_full_test = pickle.load(file)
else:
    h_eng_full_test = [desc_to_tokens(a, True) for a in tqdm(tokens_description_lemm_mod0_test)]
    with open('tokens_description_plus_lemment_mod0_eng_test.pkl', 'wb') as file:
        pickle.dump(h_eng_full_test, file)

In [1012]:
if(os.path.exists("/kaggle/input/nsu-jobs/ExpEng_test.csv")):
    EngExp_test = pd.read_csv("/kaggle/input/nsu-jobs/ExpEng_test.csv")
    X_test=X_test.merge(EngExp_test ,how='left', on='id')
    X_test.eng = X_test.eng.fillna('')
else:
    h_exp_full_test = [desc_to_tokens(a, False, False) for a in tokens_description_lemm_mod0_test]
    h1_exp_full_test = [get_experience(u) for u in h_exp_full_test]
    
    h1_eng_full_test = [get_english(t) for t in h_eng_full_test]
    
    X_test['exp'] = h1_exp_full_test
    X_test['eng'] = h1_eng_full_test
    X_test[['id','exp','eng']].to_csv('ExpEng_test.csv', index=False)
    X_train.eng = X_train.eng.fillna('')

In [1013]:
# h_eng_full_test = [desc_to_tokens(a, True) for a in tqdm(tokens_description_full_test)]
# h1_eng_full_test = [get_english(t) for t in h_eng_full_test]
# len(h_eng_full_test ),len(h1_eng_full_test)

In [1014]:
# X_test['exp'] = h1_exp_full_test
# X_test['eng'] = h1_eng_full_test
# X_test[['id','exp','eng']].to_csv('ExpEng_test.csv', index=False)

In [1015]:
# EngExp_test = pd.read_csv("/kaggle/input/nsu-jobs/ExpEng_test.csv")
# X_test=X_test.merge(EngExp_test ,how='left', on='id')
# X_test.eng = X_test.eng.fillna('')

In [1016]:
desc_is_bonus_test=[is_bonus(t) for t in tokens_description_lemm_mod0_test]
len(desc_is_bonus_test)

9312

In [1017]:
X_test['is_bonus']=desc_is_bonus_test

In [1018]:
import pickle
if(os.path.exists("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod1_test.pkl")):
    with open("/kaggle/input/nsu-jobs/tokens_description_plus_lemment_mod1_test.pkl", 'rb') as file:
        tokens_description_lemm_mod1_test = pickle.load(file)
else:
    tokens_description_lemm_mod1_test = [tokenize(d,is_lemmatization=True,mod = 1) for d in tqdm(X_test['description'])]
    with open('tokens_description_plus_lemment_mod1_test.pkl', 'wb') as file:
        pickle.dump(tokens_description_lemm_mod1_test, file)

In [1019]:
descs_tokens_rus_test = list(map(lambda desc: list(filter(lambda token: is_rus(token), desc)), tokens_description_lemm_mod1_test))
len(descs_tokens_rus_test)

9312

In [1020]:
descs_tokens_full_test = [desc_to_tokens(d) for d in descs_tokens_rus_test]

In [1021]:
oklads_full_test = [find_salary(descs_token) for descs_token in descs_tokens_full_test]

In [1022]:
salary_min_desc_full_test = []
salary_max_desc_full_test = []
for ind, oklad in enumerate(oklads_full_test):
    salary_from = X_train.salary_from[ind]
    if(oklad[1]==salary_from):
        salary_min_desc_full_test.append(oklad[1])
        salary_max_desc_full_test.append(None)
    else:
        salary_min_desc_full_test.append(oklad[0])
        salary_max_desc_full_test.append(oklad[1])

In [1023]:
X_test['salary_min_desc']=salary_min_desc_full_test
X_test['salary_max_desc']=salary_max_desc_full_test
X_test.loc[((X_test.salary_from>X_test.salary_max_desc) | (abs(X_test['salary_min_desc']-X_test.salary_from)>X_test.salary_from*0.1)),['salary_min_desc', 'salary_max_desc']] = None 

In [1024]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        9312 non-null   int64              
 1   name                      9312 non-null   object             
 2   has_test                  9312 non-null   bool               
 3   response_letter_required  9312 non-null   bool               
 4   salary_from               7924 non-null   float64            
 5   salary_currency           9312 non-null   object             
 6   salary_gross              9312 non-null   bool               
 7   published_at              9312 non-null   datetime64[ns, UTC]
 8   created_at                9312 non-null   datetime64[ns, UTC]
 9   employer_name             9312 non-null   object             
 10  description               9312 non-null   object             
 11  area_id          

In [1025]:
agg_prof1_emp_test = X_test.groupby(['start_name1','employer_name'],sort=False).agg(
    salary_mean_prof1_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_prof2_emp_test = X_test.groupby(['start_name2','employer_name'],sort=False).agg(
    salary_mean_prof2_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_prof3_emp_test = X_test.groupby(['start_name3','employer_name'],sort=False).agg(
    salary_mean_prof3_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_emp_area_test = X_test.groupby(['employer_name','area_id'],sort=False).agg(
    salary_mean_emp_area = ('salary_from', lambda x: np.mean(x)),
)
agg_prof1_area_test = X_test.groupby(['start_name1','area_id'],sort=False).agg(
    salary_mean_prof1_area = ('salary_from', lambda x: np.mean(x)),
)
agg_emp_test = X_test.groupby(['employer_name',],sort=False).agg(
    salary_mean_emp = ('salary_from', lambda x: np.mean(x)),
)
agg_prof_test = X_test.groupby(['start_name1',],sort=False).agg(
    salary_mean_prof = ('salary_from', lambda x: np.mean(x)),
)
agg_area_test = X_test.groupby(['area_id',],sort=False).agg(
    salary_mean_area = ('salary_from', lambda x: np.mean(x)),
)
agg_all_test = X_test.salary_from.mean()

agg_prof12_test = X_test.groupby(['start_name1','start_name2',],sort=False).agg(
    salary_mean_prof12 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof13_test = X_test.groupby(['start_name1','start_name3',],sort=False).agg(
    salary_mean_prof13 = ('salary_from', lambda x: np.mean(x)),
)
agg_prof23_test = X_test.groupby(['start_name2','start_name3',],sort=False).agg(
    salary_mean_prof23 = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_emp_test = X_test.groupby(['optimum_title','employer_name'],sort=False).agg(
    salary_mean_opt_emp = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_area_test = X_test.groupby(['optimum_title','area_id'],sort=False).agg(
    salary_mean_opt_area = ('salary_from', lambda x: np.mean(x)),
)

agg_opt_test = X_test.groupby(['optimum_title'],sort=False).agg(
    salary_mean_opt = ('salary_from', lambda x: np.mean(x)),
)
agg_opt_prof1_test = X_test.groupby(['optimum_title', 'start_name1'],sort=False).agg(
    salary_mean_opt_prof1 = ('salary_from', lambda x: np.mean(x)),
)

In [1026]:
agg_opt_emp_test.reset_index(inplace=True)
agg_opt_area_test.reset_index(inplace=True)
agg_opt_test.reset_index(inplace=True)
agg_opt_prof1_test.reset_index(inplace=True)

agg_prof1_emp_test.reset_index(inplace=True)
agg_prof2_emp_test.reset_index(inplace=True)
agg_prof3_emp_test.reset_index(inplace=True)
agg_emp_area_test.reset_index(inplace=True)
agg_prof1_area_test.reset_index(inplace=True)
agg_emp_test.reset_index(inplace=True)
agg_prof_test.reset_index(inplace=True)
agg_area_test.reset_index(inplace=True)
agg_prof12_test.reset_index(inplace=True)
agg_prof13_test.reset_index(inplace=True)
agg_prof23_test.reset_index(inplace=True)

agg_opt_emp_test=agg_opt_emp_test.dropna()
agg_opt_area_test=agg_opt_area_test.dropna()
agg_opt_test=agg_opt_test.dropna()
agg_opt_prof1_test=agg_opt_prof1_test.dropna()

agg_prof1_emp_test = agg_prof1_emp_test.dropna()
agg_prof2_emp_test = agg_prof2_emp_test.dropna()
agg_prof3_emp_test = agg_prof3_emp_test.dropna()
agg_emp_area_test = agg_emp_area_test.dropna()
agg_prof1_area_test = agg_prof1_area_test.dropna()
agg_emp_test = agg_emp_test.dropna()
agg_prof_test = agg_prof_test.dropna()
agg_area_test = agg_area_test.dropna()
agg_prof12_test = agg_prof12_test.dropna()
agg_prof13_test= agg_prof13_test.dropna()
agg_prof23_test = agg_prof23_test.dropna()

In [1027]:
X_test2 = X_test.copy()
without_salary_test = X_test2[X_test2.salary_from.isnull()]

In [1028]:
without_salary_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1388 entries, 10 to 9310
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        1388 non-null   int64              
 1   name                      1388 non-null   object             
 2   has_test                  1388 non-null   bool               
 3   response_letter_required  1388 non-null   bool               
 4   salary_from               0 non-null      float64            
 5   salary_currency           1388 non-null   object             
 6   salary_gross              1388 non-null   bool               
 7   published_at              1388 non-null   datetime64[ns, UTC]
 8   created_at                1388 non-null   datetime64[ns, UTC]
 9   employer_name             1388 non-null   object             
 10  description               1388 non-null   object             
 11  area_id              

In [1029]:
without_salary_test = without_salary_test.merge(agg_opt_emp_test, on=['optimum_title','employer_name'], how='left')
without_salary_test = without_salary_test.merge(agg_opt_area_test, on=['optimum_title','area_id'], how='left')
without_salary_test = without_salary_test.merge(agg_opt_test, on=['optimum_title'], how='left')
without_salary_test = without_salary_test.merge(agg_opt_prof1_test, on=['optimum_title', 'start_name1'], how='left')

without_salary_test = without_salary_test.merge(agg_prof12_test, on=['start_name1','start_name2'], how='left')
without_salary_test = without_salary_test.merge(agg_prof13_test, on=['start_name1','start_name3'], how='left')
without_salary_test= without_salary_test.merge(agg_prof23_test, on=['start_name2','start_name3'], how='left')

without_salary_test = without_salary_test.merge(agg_prof1_emp_test, on=['start_name1','employer_name'], how='left')
without_salary_test = without_salary_test.merge(agg_prof2_emp_test, on=['start_name2','employer_name'], how='left')
without_salary_test = without_salary_test.merge(agg_prof3_emp_test, on=['start_name3','employer_name'], how='left')

without_salary_test = without_salary_test.merge(agg_emp_area_test, on=['employer_name','area_id'], how='left')
without_salary_test = without_salary_test.merge(agg_prof1_area_test, on=['start_name1','area_id'], how='left')
without_salary_test = without_salary_test.merge(agg_emp_test, on=['employer_name',], how='left')
without_salary_test = without_salary_test.merge(agg_prof_test, on=['start_name1',], how='left')
without_salary_test = without_salary_test.merge(agg_area_test, on=['area_id',], how='left')

In [1030]:
from tqdm import tqdm
median_emp_test=[]
median_prof_test = []
for i in tqdm(range(len(without_salary_test))):
    row_values_emp_test = without_salary_test.loc[i, ['salary_mean_prof1_emp','salary_mean_prof2_emp', 'salary_mean_prof3_emp','salary_mean_opt_emp']]
    row_values_prof_test = without_salary_test.loc[i, ['salary_mean_opt_prof1','salary_mean_opt_area', 'salary_mean_opt','salary_mean_prof12','salary_mean_prof13','salary_mean_prof23','salary_mean_emp_area','salary_mean_prof1_area','salary_mean_emp','salary_mean_prof']]
    median_emp_test.append(row_values_emp_test.mean())
    median_prof_test.append(row_values_prof_test.mean())

100%|██████████| 1388/1388 [00:02<00:00, 521.25it/s]


In [1031]:
without_salary_test['median_emp'] = median_emp_test
without_salary_test['median_prof'] = median_prof_test
without_salary_test.loc[(without_salary_test.salary_from.isnull()),'salary_from'] = without_salary_test.loc[(without_salary_test.salary_from.isnull())].median_emp
without_salary_test.loc[(without_salary_test.salary_from.isnull()),'salary_from'] = without_salary_test.loc[(without_salary_test.salary_from.isnull())].median_prof
without_salary_test.loc[(without_salary_test.salary_from.isnull()),'salary_from'] = without_salary_test.loc[(without_salary_test.salary_from.isnull())].salary_mean_area
without_salary_test = without_salary_test.rename (columns= {'salary_from': 'salary_from_without'})

In [1032]:
without_salary_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        1388 non-null   int64              
 1   name                      1388 non-null   object             
 2   has_test                  1388 non-null   bool               
 3   response_letter_required  1388 non-null   bool               
 4   salary_from_without       1388 non-null   float64            
 5   salary_currency           1388 non-null   object             
 6   salary_gross              1388 non-null   bool               
 7   published_at              1388 non-null   datetime64[ns, UTC]
 8   created_at                1388 non-null   datetime64[ns, UTC]
 9   employer_name             1388 non-null   object             
 10  description               1388 non-null   object             
 11  area_id          

In [1033]:
X_test2 = X_test.copy()
X_test2 = X_test2.merge(agg_opt_emp_test, on=['optimum_title','employer_name'], how='left')
X_test2 =X_test2.merge(agg_opt_area_test, on=['optimum_title','area_id'], how='left')
X_test2 = X_test2.merge(agg_opt_test, on=['optimum_title'], how='left')
X_test2 =X_test2.merge(agg_opt_prof1_test, on=['optimum_title', 'start_name1'], how='left')

X_test2 = X_test2.merge(agg_prof12_test, on=['start_name1','start_name2'], how='left')
X_test2 = X_test2.merge(agg_prof13_test, on=['start_name1','start_name3'], how='left')
X_test2= X_test2.merge(agg_prof23_test, on=['start_name2','start_name3'], how='left')

X_test2 = X_test2.merge(agg_prof1_emp_test, on=['start_name1','employer_name'], how='left')
X_test2 = X_test2.merge(agg_prof2_emp_test, on=['start_name2','employer_name'], how='left')
X_test2 = X_test2.merge(agg_prof3_emp_test, on=['start_name3','employer_name'], how='left')

X_test2 = X_test2.merge(agg_emp_area_test, on=['employer_name','area_id'], how='left')
X_test2 = X_test2.merge(agg_prof1_area_test, on=['start_name1','area_id'], how='left')
X_test2 = X_test2.merge(agg_emp_test, on=['employer_name',], how='left')
X_test2 = X_test2.merge(agg_prof_test, on=['start_name1',], how='left')
X_test2 = X_test2.merge(agg_area_test, on=['area_id',], how='left')

In [1034]:
X_test2= X_test2.merge(without_salary_test[['id','salary_from_without']], on='id', how='left')
X_test2.loc[(X_test2.salary_from.isnull()),'salary_from'] = X_test2.salary_from_without

In [1035]:
X_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        9312 non-null   int64              
 1   name                      9312 non-null   object             
 2   has_test                  9312 non-null   bool               
 3   response_letter_required  9312 non-null   bool               
 4   salary_from               9312 non-null   float64            
 5   salary_currency           9312 non-null   object             
 6   salary_gross              9312 non-null   bool               
 7   published_at              9312 non-null   datetime64[ns, UTC]
 8   created_at                9312 non-null   datetime64[ns, UTC]
 9   employer_name             9312 non-null   object             
 10  description               9312 non-null   object             
 11  area_id          

In [1036]:
X_test = X_test2.drop(columns=["id", "description","salary_currency", 'published_at', 'created_at', 'salary_from_without'])

In [1037]:
X_test[X_test.loc[:, X_test.dtypes == object].columns] = X_test.loc[:, X_test.dtypes == object].astype('category')

In [1038]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9312 entries, 0 to 9311
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   name                      9312 non-null   category
 1   has_test                  9312 non-null   bool    
 2   response_letter_required  9312 non-null   bool    
 3   salary_from               9312 non-null   float64 
 4   salary_gross              9312 non-null   bool    
 5   employer_name             9312 non-null   category
 6   area_id                   9312 non-null   int64   
 7   area_name                 9312 non-null   category
 8   timestamp                 9312 non-null   float64 
 9   start_name1               9312 non-null   category
 10  start_name2               9312 non-null   category
 11  start_name3               9312 non-null   category
 12  optimum_title             9312 non-null   category
 13  exp                       9312 non-null   float6

In [1039]:
y_pred_test = cat.predict(X_test)

In [1040]:
qw_test = pd.DataFrame({ 'X_from': X_test.salary_from, 'y_pred': y_pred_test,'salary_max_desc':X_test.salary_max_desc, 'y_pred_copy': y_pred_test  })

In [1041]:
qw_test.loc[~qw_test.salary_max_desc.isnull(),'y_pred'] = qw_test.loc[~qw_test.salary_max_desc.isnull()].salary_max_desc
qw_test.loc[~qw_test.salary_max_desc.isnull()]

,X_from,y_pred,salary_max_desc,y_pred_copy
2,200000.0,200000.0,200000.0,290698.259382
21,40000.0,80000.0,80000.0,98536.288357
22,40000.0,45000.0,45000.0,48102.657307
30,80000.0,100000.0,100000.0,127281.374860
33,40000.0,55000.0,55000.0,61212.712330
...,...,...,...,...
9270,80000.0,80000.0,80000.0,114459.169787
9294,190000.0,340000.0,340000.0,312096.769406
9295,45000.0,75000.0,75000.0,58784.202051
9305,60000.0,140000.0,140000.0,131642.907420


In [1042]:
qw_test.loc[((~qw_test.salary_max_desc.isnull()) &(qw_test.y_pred>qw_test.X_from*2)  )]

,X_from,y_pred,salary_max_desc,y_pred_copy
132,84000.000000,180000.0,180000.0,153176.932842
166,15000.000000,40000.0,40000.0,32639.956017
213,95000.000000,350000.0,350000.0,280172.604822
229,63000.000000,150000.0,150000.0,158864.893190
233,30000.000000,700000.0,700000.0,40900.344719
...,...,...,...,...
8976,30000.000000,100000.0,100000.0,98963.593428
9064,32200.000000,82000.0,82000.0,45395.366109
9177,49833.844627,238000.0,238000.0,136821.168841
9195,25000.000000,80000.0,80000.0,69027.179156


In [1043]:
qw_test.loc[((~qw_test.salary_max_desc.isnull()) &(qw_test.y_pred>qw_test.X_from*2)  ), 'y_pred'] = qw_test.loc[((~qw_test.salary_max_desc.isnull()) &(qw_test.y_pred>qw_test.X_from*2))].y_pred_copy

In [1044]:
qw_test.loc[((~qw_test.salary_max_desc.isnull()) &(qw_test.y_pred>qw_test.X_from*2)  )]

,X_from,y_pred,salary_max_desc,y_pred_copy
166,15000.000000,32639.956017,40000.0,32639.956017
213,95000.000000,280172.604822,350000.0,280172.604822
229,63000.000000,158864.893190,150000.0,158864.893190
259,55000.000000,131372.879400,150000.0,131372.879400
261,40000.000000,118983.138589,120000.0,118983.138589
...,...,...,...,...
8969,70000.000000,175250.375470,235000.0,175250.375470
8976,30000.000000,98963.593428,100000.0,98963.593428
9177,49833.844627,136821.168841,238000.0,136821.168841
9195,25000.000000,69027.179156,80000.0,69027.179156


In [1045]:
qw_test['difff'] = abs(qw_test.salary_max_desc-qw_test.y_pred_copy)
qw_test[((~qw_test.salary_max_desc.isnull())&(qw_test.X_from>qw_test.y_pred )  )].sort_values(by = 'difff').tail(30)

,X_from,y_pred,salary_max_desc,y_pred_copy,difff
9052,89375.000000,10000.0,10000.0,67266.333970,57266.333970
8701,30000.000000,24000.0,24000.0,81733.386059,57733.386059
175,97859.312248,60000.0,60000.0,118423.156679,58423.156679
2861,30000.000000,24000.0,24000.0,86068.037215,62068.037215
6006,66182.892601,60000.0,60000.0,125057.233811,65057.233811
2395,75753.978004,10000.0,10000.0,75639.393456,65639.393456
8075,250000.000000,200000.0,200000.0,265707.284425,65707.284425
1416,127863.678728,100000.0,100000.0,169302.496108,69302.496108
2868,90646.057635,80000.0,80000.0,150818.660348,70818.660348
1124,151088.676559,130000.0,130000.0,203957.816672,73957.816672


In [1046]:
qw_test.loc[((~qw_test.salary_max_desc.isnull()) &(qw_test.X_from>qw_test.y_pred )& (qw_test.difff>40000)), 'y_pred'] = qw_test.loc[((~qw_test.salary_max_desc.isnull())  &(qw_test.X_from>qw_test.y_pred )& (qw_test.difff>40000))].y_pred_copy

In [1047]:
qw_test[((~qw_test.salary_max_desc.isnull()) )].sort_values(by = 'difff').tail(20)

,X_from,y_pred,salary_max_desc,y_pred_copy,difff
6957,42435.158730,64469.570407,300000.0,64469.570407,235530.429593
2981,150000.000000,257048.747623,500000.0,257048.747623,242951.252377
1128,168860.845699,281768.048709,25000.0,281768.048709,256768.048709
5032,65000.000000,221243.706625,500000.0,221243.706625,278756.293375
2706,62610.283909,177249.425643,500000.0,177249.425643,322750.574357
1349,220000.000000,320434.094145,650000.0,320434.094145,329565.905855
4994,80000.000000,192916.590631,600000.0,192916.590631,407083.409369
8418,40000.000000,92834.122740,500000.0,92834.122740,407165.877260
5178,100000.000000,213622.195032,625752.0,213622.195032,412129.804968
1163,30000.000000,85748.836130,500000.0,85748.836130,414251.163870


In [1048]:
qw_test.loc[qw_test.X_from<150]

,X_from,y_pred,salary_max_desc,y_pred_copy,difff
466,40.0,79460.045112,NaN,79460.045112,NaN
989,70.0,154670.255689,NaN,154670.255689,NaN
1393,50.0,70447.877620,NaN,70447.877620,NaN
1828,35.0,28437.373651,NaN,28437.373651,NaN
2509,60.0,9745.419542,NaN,9745.419542,NaN
2650,20.0,119204.279162,NaN,119204.279162,NaN
3330,1.0,-4883.359583,20000.0,-4883.359583,24883.359583
3524,1.0,688782.150067,NaN,688782.150067,NaN
4079,100.0,158315.610759,NaN,158315.610759,NaN
4749,60.0,63267.490013,NaN,63267.490013,NaN


In [1049]:
qw_test.loc[qw_test.X_from<150,'y_pred']  = qw_test.loc[qw_test.X_from<=150].X_from*1.2
y_pred_test=qw_test.y_pred

In [1050]:
qw_test.tail(10)

,X_from,y_pred,salary_max_desc,y_pred_copy,difff
9302,10000.000000,60349.219845,NaN,60349.219845,NaN
9303,60000.000000,77565.717281,NaN,77565.717281,NaN
9304,55000.000000,191580.896334,NaN,191580.896334,NaN
9305,60000.000000,131642.907420,140000.0,131642.907420,8357.092580
9306,60000.000000,80000.000000,80000.0,125998.543424,45998.543424
9307,40000.000000,63987.545599,NaN,63987.545599,NaN
9308,40000.000000,96231.768297,NaN,96231.768297,NaN
9309,35000.000000,39860.230385,NaN,39860.230385,NaN
9310,51071.185507,75591.708586,NaN,75591.708586,NaN
9311,30000.000000,34859.539976,NaN,34859.539976,NaN


In [1051]:
df_sub = pd.read_csv('/kaggle/input/nsu-jobs/random_result.csv')

In [1052]:
df_sub.salary_to=y_pred_test
df_sub

,id,salary_to
0,20978,35435.181135
1,3102,32904.787785
2,14731,200000.000000
3,16113,208850.278750
4,8410,149377.447786
...,...,...
9307,29490,63987.545599
9308,34298,96231.768297
9309,33921,39860.230385
9310,8886,75591.708586


In [1053]:
# нормализация
df_sub.loc[df_sub.salary_to>80000,'salary_to' ]=df_sub[df_sub.salary_to>80000].salary_to*0.925
df_sub.loc[df_sub.salary_to>250000,'salary_to' ]=df_sub[df_sub.salary_to>250000].salary_to*0.975
df_sub.loc[df_sub.salary_to>500000,'salary_to' ]=df_sub[df_sub.salary_to>500000].salary_to*0.95
df_sub.loc[df_sub.salary_to<10,'salary_to' ]=df_sub[df_sub.salary_to<10].salary_to*0.83333
df_sub

,id,salary_to
0,20978,35435.181135
1,3102,32904.787785
2,14731,185000.000000
3,16113,193186.507844
4,8410,138174.139202
...,...,...
9307,29490,63987.545599
9308,34298,89014.385675
9309,33921,39860.230385
9310,8886,75591.708586


In [1054]:
df_sub.to_csv('submission.csv', index=False)